In [1]:
import vertexai
from vertexai.language_models import TextEmbeddingModel
from annoy import AnnoyIndex
import pandas as pd

import requests
import os
from dotenv import load_dotenv
import json
from IPython.display import display, Markdown

from langchain_google_vertexai import VertexAI
from langchain_core.prompts import PromptTemplate
from langchain_core.prompts import ChatPromptTemplate, PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnableBranch

load_dotenv()

vertexai.init(project="vidio-quiz-prod", location="asia-southeast1")
embedding_model = TextEmbeddingModel.from_pretrained("textembedding-gecko-multilingual")

In [5]:
import csv
csv_file_path = 'data/nginx_data.csv'
data = ""

with open(csv_file_path, 'r') as file:
    # Create a CSV reader object
    csv_reader = csv.DictReader(file)

    headers = csv_reader.fieldnames
    
    data += ','.join(headers) + '\n'
    
    i = 0
    # Iterate over each row in the CSV file
    # data too big, need sampling
    for row in csv_reader:
        if(i % 90 == 0):
            row_str = ','.join(row.values())
            data += row_str
            data += "\n"
        
        i += 1


model = VertexAI(model_name="gemini-pro")

def tanya(query):
    prompt = ChatPromptTemplate.from_template("""
You are an on call engineer expert at reading logs and getting summarize

Here are the nginx logs:
{data} 

this is the user question: {query}
return the result as markdown
    """)
    output_parser = StrOutputParser()

    chain = prompt | model | output_parser

    result = chain.invoke({"data": data, "query": query})
    print(result)

count = (len(data))

if count > 10000:
    print(f"TOO MANY CHARS, MAX 10.000, input: {count}")
else:
    tanya("Summarize the data for me")

**Top Nginx referrer.keyword**


| Rank | Keyword | Count |
|---|---|---|
| 1 | android-app://com.vidio.android | 265 |
| 2 | androidtv-app://com.vidio.android.tv | 560 |
| 3 | ios-app://com.kmk.vidio | 312 |
| 4 | (empty) | 295 |
| 5 | https://m.vidio.com/ | 122 |
| 6 | https://www.vidio.com/ | 102 |
| 7 | https://tv.vidio.com/ | 78 |
| 8 | https://www.liputan6.com/ | 27 |
| 9 | https://www.bola.com/ | 19 |
| 10 | https://m.vidio.com/sw-moengage.js | 19 |
